<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ca5093d39bfc7a5c706da4dd9bb47ccbc2df99bc70e0b0e457865b1f630d6564
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 12:41:49
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.50 L
Current:  1.45 C
-------------------
Today PnL: 1.31 L (0.91%)
Current PnL: -21.55 L (-14.12%)
CY Booked + Current PnL: -8.03 L (-5.26%)
-------------------
Total profit:  1.88 L
Total loss:  -23.43 L
-------------------
Total Booked + Current PnL: 18.95 L (15.09%)
Total Booked PnL: 40.50 L (32.24%)
Curr Year Booked PnL: 13.52 L (9.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 88.22 L (60.8%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.39%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,101.49,51.0,M-SC,2.54,86794.0,-13983.0,14078.0,0.14,-13.88,16.22,0.09,245.0,-0.99,0.60,14.03,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199121.0,-1017.0,21903.0,0.09,-0.51,11.00,10.44,4.0,-0.05,1.39,5.03,X40,NTT,FMCG
34,ICICIGI,2252.93,-13.14,57.0,X-MC,7.55,187544.0,13224.0,21980.0,1.03,7.59,11.72,20.19,91.0,0.60,1.31,23.85,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-2.75,64.0,X-LC,8.57,299099.0,33673.0,23120.0,0.80,12.69,7.73,21.40,11.0,1.46,2.08,20.17,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,ROUTE,2227.21,-56.49,41.0,H-SC,23.18,71427.0,-61639.0,153518.0,5.24,-46.32,214.93,69.05,118.0,-0.40,0.50,5.24,SR,ATH,IT
39,INDUSINDBK,1800.00,299.86,78.0,L-MC,7.11,49066.0,-28140.0,53536.0,4.20,-36.45,109.11,32.89,258.0,-0.53,0.34,35.12,XR,NTT,BANKS
49,LTIM,7201.88,6.94,71.0,H-LC,10.18,265522.0,19371.0,58574.0,3.33,7.87,22.06,31.66,16.0,0.33,1.85,47.33,X200,ATH,IT
16,CAMS,4762.00,-2.50,58.0,X-SC,1.23,261532.0,9676.0,52751.0,3.24,3.84,20.17,24.79,122.0,0.18,1.82,27.87,X40N,NTT,MISC
72,TATAELXSI,9161.00,-23.37,53.0,H-MC,6.32,103569.0,-28266.0,70489.0,3.17,-21.44,68.06,32.03,98.0,-0.40,0.72,16.01,OX40N,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,671.64,-18.39,47.0,X-LC,5.68,293863.0,-21979.0,134648.0,-2.12,-6.96,45.82,35.67,5.0,-0.16,2.05,5.93,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
56,RAJOOENG,143.10,-45.28,37.0,H-SC,17.48,82960.0,-19540.0,60138.0,-0.87,-19.06,72.49,39.61,114.0,-0.32,0.58,1.32,AR,ATH,MISC
8,ATULAUTO,844.00,3559.46,51.0,M-SC,4.00,164390.0,-32742.0,122569.0,-0.86,-16.61,74.56,45.57,236.0,-0.27,1.14,16.55,XY24,NTT,AUTO
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,112948.0,-1850.0,95893.0,-0.82,-1.61,84.90,81.92,223.0,-0.02,0.79,33.71,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152241.0,1296.0,108152.0,1.6,0.86,71.04,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210613.0,1039.0,73630.0,1.5,0.50,34.96,35.63,NaN,0.01,1.47,17.25,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.0,101.49,51.0,M-SC,2.54,86794.0,-13983.0,14078.0,0.14,-13.88,16.22,0.09,245.0,-0.99,0.60,14.03,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,128.24,78.0,M-SC,4.47,149817.0,-24742.0,24780.0,0.47,-14.17,16.54,0.02,221.0,-1.00,1.04,30.31,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-67.97,43.0,H-SC,2.32,220491.0,-49176.0,85506.0,0.47,-18.24,38.78,13.47,138.0,-0.58,1.54,12.11,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.37,53.0,H-MC,6.32,103569.0,-28266.0,70489.0,3.17,-21.44,68.06,32.03,98.0,-0.40,0.72,16.01,OX40N,NTT,IT
66,SIS,528.0,2026.42,48.0,H-SC,2.21,85820.0,-25212.0,48823.0,0.76,-22.71,56.89,21.26,156.0,-0.52,0.60,15.95,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210613.0,1039.0,73630.0,1.50,0.50,34.96,35.63,NaN,0.01,1.47,17.25,AR,ATH,PHARMA
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152241.0,1296.0,108152.0,1.60,0.86,71.04,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
37,INDIAMART,4810.62,-52.40,61.0,H-SC,4.91,132659.0,9323.0,122298.0,2.08,7.56,92.19,106.72,119.0,0.08,0.92,32.70,AR,ATH,MISC
61,SAMMAANCAP,326.00,-192.05,55.0,M-SC,35.62,159750.0,10530.0,133647.0,0.65,7.06,83.66,96.62,208.0,0.08,1.11,72.88,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIAMART,4810.62,-52.40,61.0,H-SC,4.91,132659.0,9323.0,122298.0,2.08,7.56,92.19,106.72,119.0,0.08,0.92,32.70,AR,ATH,MISC
84,WIPRO,420.00,-13.34,55.0,M-LC,5.72,152241.0,1296.0,108152.0,1.60,0.86,71.04,72.51,53.0,0.01,1.06,6.93,XR,NTT,IT
85,ZYDUSLIFE,1286.17,-8.62,40.0,H-MC,6.00,210613.0,1039.0,73630.0,1.50,0.50,34.96,35.63,NaN,0.01,1.47,17.25,AR,ATH,PHARMA
83,WHIRLPOOL,2270.00,-45.46,36.0,M-SC,8.83,112948.0,-1850.0,95893.0,-0.82,-1.61,84.90,81.92,223.0,-0.02,0.79,33.71,XR,NTT,DURABLES
47,KPIGREEN,731.05,3.26,45.0,H-SC,10.18,122837.0,-2460.0,61382.0,0.24,-1.96,49.97,47.03,141.0,-0.04,0.86,53.80,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
26,GLAXO,3466.20,-20.84,29.0,X-MC,4.86,200024.0,740.0,77269.0,0.97,0.37,38.63,39.15,96.0,0.01,1.39,29.48,X40,ATH,PHARMA
25,GILLETTE,11206.78,-16.79,30.0,X-MC,5.71,255705.0,1809.0,80496.0,2.88,0.71,31.48,32.42,105.0,0.02,1.78,15.92,X40,ATH,FMCG
1,ABB,7934.00,-43.27,30.0,H-LC,5.04,237264.0,-24355.0,143568.0,-0.71,-9.31,60.51,45.57,7.0,-0.17,1.65,0.96,AR,NTT,ELECTRICAL
45,JSWINFRA,342.00,-23.96,32.0,X-MC,4.70,191025.0,-9322.0,39829.0,-0.14,-4.65,20.85,15.23,178.0,-0.23,1.33,26.10,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199121.0,-1017.0,21903.0,0.09,-0.51,11.00,10.44,4.0,-0.05,1.39,5.03,X40,NTT,FMCG
53,PGHH,17907.65,-32.66,33.0,X-MC,1.12,195000.0,-5820.0,73612.0,0.63,-2.90,37.75,33.76,80.0,-0.08,1.36,2.15,X40,ATH,FMCG
65,SIEMENS,4671.50,-5.82,37.0,H-LC,1.68,151230.0,-34865.0,82345.0,0.09,-18.74,54.45,25.51,15.0,-0.42,1.05,11.44,AR,ATH,ELECTRICAL
30,HAVELLS,2069.16,-8.30,41.0,X-MC,3.29,305886.0,-22105.0,128625.0,0.90,-6.74,42.05,32.48,92.0,-0.17,2.13,3.92,X40,ATH,ELECTRICAL
78,UNITDSPR,1644.00,-5.95,63.0,X-LC,3.44,246648.0,12504.0,36109.0,1.38,5.34,14.64,20.77,86.0,0.35,1.72,12.63,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,33.0,X-SC,37.13,48208.0,-16798.0,167725.0,0.75,-25.84,347.92,232.18,198.0,-0.10,0.34,0.75,XY24,NTT,MISC
1,ABB,7934.00,-43.27,30.0,H-LC,5.04,237264.0,-24355.0,143568.0,-0.71,-9.31,60.51,45.57,7.0,-0.17,1.65,0.96,AR,NTT,ELECTRICAL
53,PGHH,17907.65,-32.66,33.0,X-MC,1.12,195000.0,-5820.0,73612.0,0.63,-2.90,37.75,33.76,80.0,-0.08,1.36,2.15,X40,ATH,FMCG
10,BAJAJHFL,181.50,-15.96,39.0,X-MC,6.32,177235.0,-23271.0,120431.0,0.36,-11.61,67.95,48.45,90.0,-0.19,1.23,2.20,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,33.0,X-SC,37.13,48208.0,-16798.0,167725.0,0.75,-25.84,347.92,232.18,198.0,-0.10,0.34,0.75,XY24,NTT,MISC
58,RELAXO,1176.00,-40.17,45.0,X-SC,3.70,81498.0,-63662.0,139590.0,0.37,-43.86,171.28,52.31,136.0,-0.46,0.57,9.09,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.74,86860.0,-41810.0,87112.0,-0.04,-32.49,100.29,35.20,219.0,-0.48,0.60,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-11.06,43.0,X-MC,3.32,87810.0,-2868.0,17773.0,-0.95,-3.16,20.24,16.44,101.0,-0.16,0.61,16.37,X40,ATH,PHARMA
33,HONAUT,58357.33,-24.23,37.0,X-SC,0.93,106785.0,-21153.0,68289.0,0.25,-16.53,63.95,36.84,143.0,-0.31,0.74,9.32,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-26.44,63.0,X-LC,8.95,293346.0,-52610.0,119304.0,2.42,-15.21,40.67,19.28,1.0,-0.44,2.04,8.45,X40,ATH,IT
40,INFY,2275.00,-15.29,67.0,X-LC,4.54,330597.0,17709.0,153992.0,1.42,5.66,46.58,54.87,3.0,0.11,2.30,14.41,X40,BTT,IT
42,ITC,452.00,-38.34,46.0,X-LC,0.82,199121.0,-1017.0,21903.0,0.09,-0.51,11.00,10.44,4.0,-0.05,1.39,5.03,X40,NTT,FMCG
81,VBL,671.64,-18.39,47.0,X-LC,5.68,293863.0,-21979.0,134648.0,-2.12,-6.96,45.82,35.67,5.0,-0.16,2.05,5.93,X40N,ATH,FMCG
1,ABB,7934.00,-43.27,30.0,H-LC,5.04,237264.0,-24355.0,143568.0,-0.71,-9.31,60.51,45.57,7.0,-0.17,1.65,0.96,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.00,299.86,78.0,L-MC,7.11,49066.0,-28140.0,53536.0,4.20,-36.45,109.11,32.89,258.0,-0.53,0.34,35.12,XR,NTT,BANKS
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,26.24,76781.0,-36768.0,76819.0,0.23,-32.38,100.05,35.27,268.0,-0.48,0.53,87.92,XR,NTT,HOTELS
7,ASIANTILES,137.00,7550.00,67.0,L-SC,14.98,83945.0,-9865.0,86480.0,-0.03,-10.52,103.02,81.67,269.0,-0.11,0.58,62.56,XR,NTT,CERAMICS
50,MASFIN,398.61,-18.70,51.0,H-SC,7.89,92265.0,-5715.0,27320.0,1.69,-5.83,29.61,22.05,152.0,-0.21,0.64,34.11,XR,ATH,FINANCE
47,KPIGREEN,731.05,3.26,45.0,H-SC,10.18,122837.0,-2460.0,61382.0,0.24,-1.96,49.97,47.03,141.0,-0.04,0.86,53.80,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,VAIBHAVGBL,521.00,71.35,60.0,H-SC,6.61,148558.0,-34217.0,145290.0,2.27,-18.72,97.80,60.77,125.0,-0.24,1.03,33.79,XR,NTT,JEWELLERY
7,ASIANTILES,137.00,7550.00,67.0,L-SC,14.98,83945.0,-9865.0,86480.0,-0.03,-10.52,103.02,81.67,269.0,-0.11,0.58,62.56,XR,NTT,CERAMICS
38,INDIGOPNTS,1408.00,128.24,78.0,M-SC,4.47,149817.0,-24742.0,24780.0,0.47,-14.17,16.54,0.02,221.0,-1.00,1.04,30.31,OX40N,NTT,PAINTS
67,SONACOMS,806.63,-29.95,70.0,M-SC,6.17,87220.0,-13939.0,53937.0,1.76,-13.78,61.84,39.54,202.0,-0.26,0.61,22.99,AR,ATH,AUTO
5,ANGELONE,3033.00,23.98,70.0,X-SC,6.89,219268.0,28262.0,29448.0,1.23,14.80,13.43,30.21,157.0,0.96,1.53,38.77,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,44.57
2,50,76.26


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.56
LC    27.92
MC    27.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.53
X40      20.71
X40N     12.41
XY25     10.17
XR        9.72
AR        9.15
OX40N     6.57
X200      1.85
SR        1.03
MH        0.86
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.99
X-MC    22.89
X-LC    19.93
M-SC    12.53
X-SC     5.60
H-LC     4.55
H-MC     3.82
M-LC     2.45
L-SC     1.44
L-LC     0.99
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.88,-3.94,37.56
IT,12.81,-14.36,75.58
FINANCE,12.13,-8.47,57.64
MISC,7.70,-17.55,76.84
PAINTS,5.76,-11.59,28.23
ELECTRICAL,5.57,-12.79,52.78
INSURANCE,4.47,1.33,33.45
PHARMA,3.47,-0.22,33.84
AUTO,3.22,-50.31,112.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213411.0,20
XR,1261598.0,13
AR,1259117.0,10
X40,994046.0,14
X40N,687417.0,9
OX40N,540829.0,9
XY25,473083.0,7
SR,272218.0,2
MH,61382.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3390179.0,24
X-MC,1399033.0,16
M-SC,1245326.0,15
X-LC,792935.0,11
X-SC,544915.0,6
H-MC,324088.0,3
H-LC,284487.0,3
L-SC,252550.0,3
M-LC,153223.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167328.0      6
           AR         837157.0      5
           XR         793344.0      7
M-SC       XY24       723656.0      6
X-MC       XY24       563826.0      4
           X40        490519.0      8
X-LC       X40        416415.0      5
X-SC       X40N       290078.0      4
H-SC       SR         272218.0      2
           OX40N      258750.0      3
X-LC       X40N       237079.0      3
H-LC       AR         225913.0      2
X-MC       XY25       184428.0      2
H-MC       XY24       179969.0      1
X-SC       XY24       167725.0      1
L-SC       XR         163299.0      2
X-MC       X40N       160260.0      2
M-SC       XR         143267.0      2
           XY25       133647.0      1
           AR         122417.0      2
           OX40N      122339.0      4
M-LC       XR         108152.0      1
L-SC       OX40N       89251.0      1
X-SC       X40         87112.0      1
H-MC       AR          73630.0      1
X-LC       XY25        73090.0      2
H-MC       OX40N       70489.0      1
X-LC       XY24        66351.0      1
H-SC       MH          61382.0      1
H-LC       X200        58574.0      1
L-MC       XR          53536.0      1
M-LC       XY25        45071.0      1
L-LC       XY25        36847.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
